In [28]:
# I need to learn this thing....
from typing import Dict, TypedDict, Optional

class GraphState(TypedDict):
    question: Optional[str] = None
    classification: Optional[str] = None
    response: Optional[str] = None

from langgraph.graph import StateGraph, END
workflow = StateGraph(GraphState) # Ok, so stategraph's state is TypedDict and everything can be configured different, no limitation in that, good


In [29]:
def classify(str):
    return 'shit'

def classify_input_node(state):
    question = state.get('question', '').strip() # what does .strip do again? 
    classification = classify(question)
    return {"classification": classification}

def handle_greeting_node(state):
    return {"response": "Hello! How can I help you today?"}

def handle_search_node(state):
    question = state.get('question', '').strip()
    search_result = f"Search result for '{question}'"
    return {"response": search_result}

In [30]:
# Alright, so it appears that the State here is passed on many many time, and everybody is somehow manipulating on the same State here. 
# Node and edges probbaly has the relationship between a computation unit, and some router unit which direct the current computation result to the next computation unit. 
# But the point seems to be that the computation node should return only attribute existing in the state itself. 
workflow.add_node("classify_input", classify_input_node)
workflow.add_node("handle_greeting", handle_greeting_node)
workflow.add_node("handle_search", handle_search_node)


In [31]:
def decide_next_node(state):
    # return "handle_greeting" if state.get('classification') == "greeting" else "handle_search"
    return "hg" if state.get('classification') == "greeting" else "hs"


workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
        "hg": "handle_greeting",
        "hs": "handle_search"
    } # I imagine this one is not redundant only if the output of the decide_next_node function returns some value that is not a state attribute
)

In [32]:
workflow.set_entry_point("classify_input")
workflow.add_edge('handle_greeting', END)
workflow.add_edge('handle_search', END)
app = workflow.compile()

In [33]:
from langchain_core.messages import HumanMessage
# inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
inputs = {"question": "Hello how are you?"}
app.invoke(inputs)

{'question': 'Hello how are you?',
 'classification': 'shit',
 'response': "Search result for 'Hello how are you?'"}